<a href="https://colab.research.google.com/github/arthur-b-martins/AI-Agents-Lamia/blob/main/pratica-2/groq_agent_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
import re
os.environ['GROQ_API_KEY'] = "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"

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast cars",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

The thrill of the open road and the rush of adrenaline! While not everyone may share the same enthusiasm, there are several reasons why fast cars hold a special place in the hearts of many:

1. **Adrenaline rush**: Fast cars provide an exhilarating experience, releasing a rush of adrenaline and endorphins that can be addictive. The thrill of acceleration, cornering, and braking is unmatched, making it a thrilling experience.
2. **Sense of freedom**: Fast cars offer a sense of liberation, allowing drivers to feel unencumbered by traffic or road constraints. This emotional high can be empowering, making drivers feel more confident and in control.
3. **Innovation and technology**: Fast cars push the boundaries of innovation, driving technological advancements, and improving overall road safety. High-performance vehicles often feature cutting-edge safety features, such as advanced aerodynamics, and improved braking systems.
4. **Social status**: Fast cars are often seen as status symbols, 

In [ ]:

# aqui é a classe agente com os metodos que ele faz
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      #inicializa com o system prompt
      self.messages.append({"role": "system", "content": self.system})

  # quando chamado ele anexa o prompt e executa
  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  # executar é basicamente chamar a LLM para responder o prompt
  def execute(self):
    completion = client.chat.completions.create(
        messages=self.messages,
        model="llama3-70b-8192"
    )
    return completion.choices[0].message.content



In [ ]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [ ]:
def calculate(operation: str) -> float:
    return eval(operation)

# aqui eu limpei o código do video
PLANET_MASSES = {
        "mercury": 3.285e23,
        "venus": 4.867e24,
        "earth": 5.972e24,
        "mars": 6.39e23,
        "jupiter": 1.898e27,
        "saturn": 5.683e26,
        "uranus": 8.681e25,
        "neptune": 1.024e26
    }

def get_planet_mass(planet: str) -> float | None:
    return PLANET_MASSES.get(planet.lower())


In [ ]:
neil_tyson = Agent(client=client, system=system_prompt)

In [ ]:
res = neil_tyson("what is the mass of venus?")
print(res)

Thought: I need to find the mass of Venus


In [ ]:

res = neil_tyson.messages
print(res)

[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944×10e25\n\nIf you have the a

In [ ]:
def agent_loop(max, system, query):
  agent = Agent(client, system)
  next_prompt = query
  tools = ['calculate', 'get_planet_mass']

  for i in range(max):
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result:
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
      chosen_tool = action[0][0]
      arg = action[0][1]

      if chosen_tool in tools:
        tool_response = eval(f"{chosen_tool}('{arg}')")
        next_prompt = f"Observation: {tool_response}"

      else:
        next_prompt = "Observation: tool not found"

      print(next_prompt)
      continue


    if "Answer" in result:
      break

    # limpar o next_prompt caso o agente divida as respostas
    if "Thought" in result:
      next_prompt = ""

In [ ]:
agent_loop(10, system_prompt, "What is the mass of mercury times 3?")

Thought: I need to find the mass of Mercury
.

Action: get_planet_mass: mercury
PAUSE
Observation: 3.285e+23
Thought: I need to multiply this by 3

Action: calculate: 3.285e+23 * 3
PAUSE
Observation: 9.854999999999999e+23
Thought: I have the answer
!

Answer: The mass of mercury times 3 is 9.854999999999999e+23.
